In [1]:
import json
import urllib.request
import time
from datetime import timedelta
import boto3
import logging

filename = 'configWrangle.json'

with open(filename, "r") as jsonFile:
    data = json.load(jsonFile)

# data["link"] = "New link1"
state = data['state']
team = data['team']
AWSAccess = data['AWSAccess']
AWSSecret = data['AWSSecret']



In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logfile = time.strftime("%Y-%m-%d_%H:%M:%S" + ".log")

# create a file handler
handler = logging.FileHandler(logfile)
handler.setLevel(logging.INFO)
# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(handler)

logger.info('Hello!!!!')

INFO:__main__:Hello!!!!


In [3]:
# create connection to Amazon s3
client = boto3.client('s3',
                      aws_access_key_id=AWSAccess, 
                      aws_secret_access_key=AWSSecret)
s3 = boto3.resource('s3',
                   aws_access_key_id=AWSAccess, 
                      aws_secret_access_key=AWSSecret)

In [4]:
import botocore
import fnmatch
import sys

BucketName = 'Team10_NV_Assignment1'
bucket = s3.Bucket(BucketName)
rawfilename = ''

# find the newest raw file in bucket and download the file to local system
for obj in bucket.objects.all():
    if fnmatch.fnmatch(obj.key, '*_WBAN_53138.csv'):
        if obj.key > rawfilename:
            rawfilename = obj.key
if rawfilename == '':
    logger.info('the raw data does not exist! errors!')
    sys.exit("the raw data does not exist! errors!")
cleanfilename = state + '_' + time.strftime("%d%m%Y") + '_WBAN_53138_clean.csv'


In [5]:
# clean data in file (cleanfilename)


In [6]:
# check clean file exists or not
flag = False
for obj in bucket.objects.all():
    if obj.key == cleanfilename:
        flag = True

# if it exists  do not do anything
# if it does not exist just upload the file
if flag == False:
    logger.info(cleanfilename + ' does not exist in bucket')
    s3.meta.client.download_file(BucketName, rawfilename, cleanfilename)
    logger.info('Downloading raw file from bucket to local system')
    #clear data
    logger.info('Cleaning data!')
    
    s3.meta.client.upload_file(cleanfilename, BucketName, cleanfilename)
    logger.info('upload cleaned file to bucket')
else:
    logger.info(cleanfilename + 'exist in bucket!')
# delete the old one
# oldfile = ''
# for obj in bucket.objects.all():
#     if fnmatch.fnmatch(obj.key, '*_WBAN_53138_clean.csv'):
#         oldfile = obj.key
# print(oldfile)
# client.delete_object(Bucket=BucketName, Key=oldfile)

# upload new file


INFO:__main__:NV_22062017_WBAN_53138_clean.csv does not exist in bucket
INFO:__main__:Downloading raw file from bucket to local system
INFO:__main__:Cleaning data!
INFO:__main__:upload cleaned file to bucket


In [7]:
# update links in json
data['rawData'] = 'https://s3.amazonaws.com/' + BucketName + '/' + rawfilename
data['cleanData'] = 'https://s3.amazonaws.com/' + BucketName + '/' + cleanfilename
with open(filename, "w") as jsonFile:
    json.dump(data, jsonFile)